# Analyzing Google Location Data

This code accompanies the ShmooCon 2018 talk: "OK Google, Tell Me About Myself."

With this code, I attempt to infer information about my daily life (i.e., identify significant locations in my life such as home, when I am most likely to be home) using a Location History file downloaded from https://takeout.google.com/settings/takeout . I specifically downloaded the Location History file in KML (Keyhole Markup Language) format.

### Initialize imports for the code

Set up all your imports for this code. You may need to install some of these packages using 'pip' before they will import successfully.

In [ ]:
import re, sys

import xml.etree.ElementTree
from dateutil import parser
from datetime import datetime
from dateutil import tz

import geopy
import geopy.distance

from sklearn.cluster import KMeans

import numpy
import statistics, math

### Import the data, clean it, and store relevant data to a data structure.

Below are several general-purpose methods that are used in this notebook.

In [ ]:
# Returns distance between two lat/long points, in miles.
def calculateDistance(lat1, long1, lat2, long2):
    return float("{0:.2f}".format(geopy.distance.distance((lat1, long1), (lat2, long2)).miles))

In [ ]:
# Note: beginDate and endDate should only contain ints
def fallsWithinTimePeriod(thisYear, thisMonth, thisDay, beginDate, endDate):
    beginYear = beginDate[0]
    endYear = endDate[0]

    if thisYear >= beginYear and thisYear <= endYear:
        if thisYear != beginYear and thisYear != endYear:
            return True
        
        # 3/17/18 Edited
        if beginYear != endYear:
            if thisYear == beginYear:
                if thisMonth > beginDate[1]:
                    return True
            
                if thisMonth == beginDate[1] and thisDay >= beginDate[2]:
                    return True
                
                return False
            
            if thisYear == endYear:
                if thisMonth < endDate[1]:
                    return True
            
                if thisMonth == endDate[1] and thisDay <= endDate[2]:
                    return True
            
                return False
        else:
            if thisMonth > beginDate[1] and thisMonth < endDate[1]:
                return True
            
            if beginDate[1] == endDate[1]:
                if thisMonth == beginDate[1] and thisDay >= beginDate[2] and thisDay <= endDate[2]:
                    return True
                
            if thisMonth == beginDate[1] and thisDay >= beginDate[2]:
                return True
            
            if thisMonth == endDate[1] and thisDay <= endDate[2]:
                return True
                
    return False

In [ ]:
def calculateSecsElapsed(timeStart, timeEnd):
    # expected time format: 12:34:56

    hourStart = int(timeStart[0:2])
    minStart = int(timeStart[3:5])
    secStart = int(timeStart[6:])
    
    hourEnd = int(timeEnd[0:2])
    minEnd = int(timeEnd[3:5])
    secEnd = int(timeEnd[6:])

    # Convert everything to seconds and then subtract
    totalStartSecs = (hourStart * 3600) + (minStart * 60) + secStart
    totalEndSecs = (hourEnd * 3600) + (minEnd * 60) + secEnd
        
    result = totalEndSecs - totalStartSecs
    
    if result < 0:
        # because of day turning over
        result = (86400 - totalStartSecs) + totalEndSecs 
        
    return result


In [ ]:
# point is tuple: (lat, long)
# boundingPoints is an array: [ min lat, max lat, min long, max long ]
def pointWithinBoundingBox(point, boundingPoints):
    pointLat = float(point[0])
    pointLong = float(point[1])
    
    minBoundingLat = float(boundingPoints[0])
    maxBoundingLat = float(boundingPoints[1])
    minBoundingLong = float(boundingPoints[2])
    maxBoundingLong = float(boundingPoints[3])
    
    return (pointLat >= minBoundingLat) and (pointLat <= maxBoundingLat) and (pointLong >= minBoundingLong) and (pointLong <= maxBoundingLong)
    

Specify information about the input file, including:
 - the direct path to the .kml file
 - the time zone (to_zone) that you want to convert the time to (from_zone should always be UTC)
 - begin date of the entire data set that you want to start analyzing
 - end date of the entire data set that you want to end analyzing
      - I used about 2 years' worth of data. Depending on your computing power, more data could take a while to cluster.

In [ ]:
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')

kmlfilename = 'path/to/kml/file'

# Date format is [ int year, int month, int day ]
beginDate = [ 2018, 9, 20 ]
endDate = [ 2019, 9, 20]

Parse the KML file and save off parallel lists of location and time data.

In [ ]:
whenList = list()
locationList = list()

# Assume "when" comes first, and the "gx:coord" immediately following it is the applicable location
def storeData(node):
    for child in node:
        if child.tag.endswith("when"):
            utc = datetime.strptime(str(parser.parse(child.text)), '%Y-%m-%d %H:%M:%S+00:00')
            utc = utc.replace(tzinfo=from_zone)
            
            # Convert time zone
            easternTime = utc.astimezone(to_zone)
            whenList.append(easternTime)
        elif child.tag.endswith("coord"):
            locationList.append(child.text)
        storeData(child)


root = xml.etree.ElementTree.parse(kmlfilename).getroot()
storeData(root)


The following method extracts information from a KML date string and stores it in a data structure, which is returned.

Note that 'dayOfWeek' starts at 0, which is Monday.

In [ ]:
def getDateTimeDict(thisDay):
    
    timeInfo = dict()

    dayOfWeek = thisDay.weekday()
    year = thisDay.year
    month = thisDay.month
    day = thisDay.day
    
    hour = str(thisDay.hour)
    if len(hour) == 1:
        hour = '0' + hour
        
    minute = str(thisDay.minute)
    if len(minute) == 1:
        minute = '0' + minute
    
    second = str(thisDay.second)
    if len(second) == 1:
        second = '0' + second
    
    time = hour + ":" + minute + ":" + second
    
    timeInfo['year'] = year
    timeInfo['month'] = month
    timeInfo['day'] = day
    timeInfo['time'] = time
    # Monday is day 0 --> different from previously-used numbering scheme!
    timeInfo['dayOfWeek'] = dayOfWeek
   
    return timeInfo

Store parallel lists of distances, times, and dates. For example, listOfLatLongs[3], listTimes[3], and listOfDates[3] all represent information for the same recorded data point. These parallel lists are used for clustering.

The 'sortedLocations' variable represents a dict (hash) of:
    - date -> information about that date
      - hash of times for that date -> time-specific information (i.e., lat/long for that time) 

In [ ]:
listOfLatLongs = list()
listOfTimes = list()
listOfDates = list()

uniqueDates = set()
sortedLocations = dict()

# Iterate backwards so that we are going forward in time in the final data structure
for i in range(len(locationList) - 1, -1, -1):
    currLoc = locationList[i]
    currDateTime = whenList[i]
    actualTimeInfo = getDateTimeDict(currDateTime)

    thisYear = str(actualTimeInfo['year'])
    thisMonth = str(actualTimeInfo['month'])
    thisDay = str(actualTimeInfo['day'])
    
    if fallsWithinTimePeriod(int(thisYear), int(thisMonth), int(thisDay), beginDate, endDate):
        tempdict = dict()
        
        distanceSplit = currLoc.split()
        if len(distanceSplit) != 3:
            print("Expected only 3 elements from split! --> " + str(distanceSplit))
            sys.exit(1)
            
        currLat = distanceSplit[1]
        currLong = distanceSplit[0]
        currAltitude = distanceSplit[2]
        
        tempdict['point'] = (float(currLat), float(currLong))
        tempdict['altitude'] = float(currAltitude)
        
        monthStr = thisMonth
        dayStr = thisDay
        
        if len(monthStr) == 1:
            monthStr = '0' + monthStr
            
        if len(dayStr) == 1:
            dayStr = '0' + dayStr
        
        dateStr = thisYear + '|' + monthStr + '|' + dayStr
        timeStr = str(actualTimeInfo['time'])
        
        listOfLatLongs.append((currLat, currLong))
        listOfTimes.append(actualTimeInfo)
        uniqueDates.add(dateStr)        
        listOfDates.append(dateStr)
    
        if dateStr not in sortedLocations:
            sortedLocations[dateStr] = dict()
            sortedLocations[dateStr]['timeInfo'] = dict()
            sortedLocations[dateStr]['year'] = thisYear
            sortedLocations[dateStr]['month'] = thisMonth
            sortedLocations[dateStr]['day'] = thisDay
            sortedLocations[dateStr]['dayOfWeek'] = actualTimeInfo['dayOfWeek']
            
        if timeStr not in sortedLocations[dateStr]:
            sortedLocations[dateStr]['timeInfo'][timeStr] = tempdict
                


In [ ]:
print(listOfDates[0])
print(listOfDates[-1])

The next few steps are for data cleaning. Check if successive points could possible be invalid -- for example, if I moved 100 miles in 3 seconds. That's obviously not possible and should be chucked (implies incorrect location reading). Store bad data points for removal in the next step.

Adjust 'maxMilePerHour' variable if you wish.

In [ ]:
maxMilePerHour = 80.0
removePoints = dict()
removeDates = list()

for date in sorted(sortedLocations.keys()):
    dateInfo = sortedLocations[date]
    
    # Start comparison calculations over when date rolls over
    prevTime = ''
    prevLatLong = (0, 0)
    
    issuesPerDate = 0

    for time in sorted(dateInfo['timeInfo'].keys()):        
        timeInfo = dateInfo['timeInfo'][time]      
        currLatLong = timeInfo['point']
        
        # Calculate rate of travel
        if len(prevTime) > 0:
            secsBetween = calculateSecsElapsed(prevTime, time)
            distBetween = calculateDistance(prevLatLong[0], prevLatLong[1], currLatLong[0], currLatLong[1])
            
            currMph = distBetween / (float(secsBetween) / 3600)
            
            if currMph > maxMilePerHour:
                # Give some leeway if more than a few minutes have elapsed. In a few minutes, it's
                # slightly plausible that I could have moved a significant distance (i.e., in a vehicle).
                if secsBetween <= 120:
                    if date not in removePoints:
                        removePoints[date] = list()
                    removePoints[date].append(time)
                    issuesPerDate += 1
                    continue
                    
        prevTime = time
        prevLatLong = currLatLong
        
    # If there were more than 100 problematic points on this day, just remove the whole day from the set
    if issuesPerDate > 100:
        removeDates.append(date)

    

Remove the problematic data points from the data structures.

In [ ]:
for dateToRemove in removeDates:
    if dateToRemove in sortedLocations:
        del sortedLocations[dateToRemove]
        
    if dateToRemove in removePoints:
        del removePoints[dateToRemove]
        
    # Find the indices for this date 
    dateIndices = [i for i, j in enumerate(listOfDates) if j == dateToRemove]    
    uniqueDates.remove(dateToRemove)        

    for dateIndex in reversed(dateIndices):
        del listOfLatLongs[dateIndex]
        del listOfTimes[dateIndex]
        del listOfDates[dateIndex]
        
for (dateForPoint, pointTimes) in removePoints.items():
    dateIndices = [i for i, j in enumerate(listOfDates) if j == dateForPoint]

    for time in pointTimes:
        del sortedLocations[dateForPoint]['timeInfo'][time]
            
        for dateIndex in reversed(dateIndices):
            if listOfTimes[dateIndex] == time:
                del listOfTimes[dateIndex]
                del listOfLatLongs[dateIndex]
                del listOfDates[dateIndex]
                break


In [ ]:
for i in range(0,3):
    print(listOfTimes[i])
    
print(listOfTimes[-1])
print(len(listOfTimes))

Now we have clean data and are ready to start looking deeper at the data!

### Clustering Points

First, we separate our points into weekday day, weekday evening, and weekend points. It should be easier to identify certain types of locations (home, work) when looking at the clusters obtained from these separated points.

In [ ]:
# NOTE: Monday is day 0, Sunday is day 6
weekdays = [ 0, 1, 2, 3, 4 ]

# Conservatively assume work hours are 7A - 7P
eveningHours = [ "00", "01", "02", "03", "04", "05", "06", "19", "20", "21", "22", "23" ]
wholeHour = range(0,60)
eveningMins = [ wholeHour, wholeHour, wholeHour, wholeHour, wholeHour, wholeHour, wholeHour, 
               wholeHour, wholeHour, wholeHour, wholeHour, wholeHour ]

weekday_day_times = list()
weekday_evening_times = list()
weekend_times = list()

weekday_day_locs = list()
weekday_evening_locs = list()
weekend_locs = list()

weekday_day_dates = list()
weekday_evening_dates = list()
weekend_dates = list()

for i in range(0, len(listOfTimes)):
    timepoint = listOfTimes[i]
    locpoint = listOfLatLongs[i]
    datepoint = listOfDates[i]

    if timepoint['dayOfWeek'] in weekdays:
        currTime = timepoint['time']
        currHour = currTime[0:2]
        currMin = int(currTime[3:5])
        
        if currHour in eveningHours:
            hourIndex = eveningHours.index(currHour)
            if currMin in eveningMins[hourIndex]:
                weekday_evening_times.append(timepoint)
                weekday_evening_locs.append(locpoint)
                weekday_evening_dates.append(datepoint)
            else:
                weekday_day_times.append(timepoint)
                weekday_day_locs.append(locpoint)
                weekday_day_dates.append(datepoint)
        else:
            weekday_day_times.append(timepoint)
            weekday_day_locs.append(locpoint)
            weekday_day_dates.append(datepoint)
    else:
        weekend_times.append(timepoint)
        weekend_locs.append(locpoint)
        weekend_dates.append(datepoint)        

Now, get and store some information from the separated points, like number of unique dates in each sub-dataset. This comes int oplay later, when I'm trying to figure out how often I appear in a specific location.

In [ ]:
uniqueWeekdayDates = list()

for date in weekday_day_dates:
    if date not in uniqueWeekdayDates:
        uniqueWeekdayDates.append(date)
        
uniqueWeekdayEveningDates = list()

for date in weekday_evening_dates:
    if date not in uniqueWeekdayEveningDates:
        uniqueWeekdayEveningDates.append(date)
        
uniqueWeekendDates = list()

for date in weekend_dates:
    if date not in uniqueWeekendDates:
        uniqueWeekendDates.append(date)
        
numUniqueWeekendDates = len(uniqueWeekendDates)
numUniqueWeekdayDayDates = len(uniqueWeekdayDates)
numUniqueWeekdayEveningDates = len(uniqueWeekdayEveningDates)

numUniqueDates = float(len(uniqueDates))

This is a helper method for clustering. This returns the max distance of each of the points in the cluster to the centroid of the cluster..

In [ ]:
def calculateMaxClusterPointDistance(center, clusterPoints):
    pointDistances = list()

    for point in clusterPoints:
        pointDistances.append(calculateDistance(center[0], center[1], point[0], point[1]))
        
    return max(pointDistances)
    

This is a helper method for clustering. This determines which point to use as the "representative" point of the cluster. If there is an actual point in the cluster that is repeated many times, then use that. Otherwise, use the calculated centroid point (calculated by the K-Means algorithm).

In [ ]:
def calculateMajorityPoint(listOfLatLongs):
    counts = dict()
    
    for latlong in listOfLatLongs:        
        if latlong not in counts:
            counts[latlong] = 1
        else:
            counts[latlong] += 1
            
    sortedCounts = sorted(counts, key=counts.get, reverse=True)

    majorityCount = counts[sortedCounts[0]]

    # Highest count should be a significant fraction of the entire data
    fractionOfData = float(majorityCount)/float(len(listOfLatLongs))
    
    if fractionOfData < 0.25:
        return (0, 0)
    
    # Ensure the result is a point made of floats (not strings)
    return (float(sortedCounts[0][0]), float(sortedCounts[0][1]))


This method is the actual workhorse of the clustering step that performs the recursive call to the K-Means algorithm.

In [ ]:
def getSignificantClusters(listOfLatLongs, listOfTimes, listOfDates, majorityPointToCount, majorityPointToDays, majorityPointUniques, majorityPointCentroids, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(listOfLatLongs)
    kmeans_predict = kmeans.predict(listOfLatLongs)
 
    for i in range(0, num_clusters):
        subLatLongList = list()
        subTimesList = list()
        subDatesList = list()
        subDatesSet = set()
        
        for j in range(0, len(kmeans_predict)):
            if kmeans_predict[j] == i:
                subLatLongList.append(listOfLatLongs[j])
                subTimesList.append(listOfTimes[j])
                subDatesList.append(listOfDates[j])
        
        lenOfList = len(subLatLongList)
 
        distance = calculateMaxClusterPointDistance(kmeans.cluster_centers_[i], subLatLongList)
        if distance <= clusterDistanceMax:
            if lenOfList >= clusterSizeMin:
                majorityPoint = calculateMajorityPoint(subLatLongList)
                
                # If there was no clear majority, the method returned (0, 0)
                if majorityPoint == (0, 0):
                    majorityPoint = tuple(kmeans.cluster_centers_[i])
                    majorityPoint = (float(majorityPoint[0]), float(majorityPoint[1]))
                
                majorityPointToCount[majorityPoint] = lenOfList
                majorityPointCentroids[majorityPoint] = kmeans.cluster_centers_[i]
                
                subDatesSet = set(subDatesList)
                majorityPointToDays[majorityPoint] = subDatesSet
                
                uniqueLatLongs = list()
                for slll in subLatLongList:
                    if slll not in uniqueLatLongs:
                        uniqueLatLongs.append(slll)
                majorityPointUniques[majorityPoint] = uniqueLatLongs
        else:
            # Only keep going if the cluster size is somewhat reasonable
            if lenOfList >= clusterSizeMin:
                getSignificantClusters(subLatLongList, subTimesList, subDatesList, majorityPointToCount, majorityPointToDays, majorityPointUniques, majorityPointCentroids, num_clusters)
    

Adjust parameters for the clustering step, if desired. Since the K-Means step is recursive, making 'clusterDistanceMax' pretty small helps to insure that the algorithm identifies locations that are close to each other (instead of including them both in the same cluster).

Try to keep the number of clusters fairly low, since the recursion will take care of subdividing too-large clusters for you.

In [ ]:
# Minimum number of points for an acceptable cluster
clusterSizeMin = 50

# Maximum radius of a cluster (in miles)
clusterDistanceMax = 0.1

numberOfClusters = 3

Create data structures to store the output of the clustering step.
Run the clustering step.

In [ ]:
weekendClusters = dict()
weekendClusterDays = dict()
weekendClusterUniquePoints = dict()
weekendClusterCentroids = dict()

weekdayDayClusters = dict()
weekdayDayClusterDays = dict()
weekdayDayClusterUniquePoints = dict()
weekdayDayClusterCentroids = dict()

weekdayEveningClusters = dict()
weekdayEveningClusterDays = dict()
weekdayEveningClusterUniquePoints = dict()
weekdayEveningClusterCentroids = dict()

print("Getting weekend clusters...")
getSignificantClusters(weekend_locs, weekend_times, weekend_dates, weekendClusters, weekendClusterDays, weekendClusterUniquePoints, weekendClusterCentroids, numberOfClusters)

print("Getting weekday evening clusters...")
getSignificantClusters(weekday_evening_locs, weekday_evening_times, weekday_evening_dates, weekdayEveningClusters, weekdayEveningClusterDays, weekdayEveningClusterUniquePoints, weekdayEveningClusterCentroids, numberOfClusters)

print("Getting weekday day clusters...")
getSignificantClusters(weekday_day_locs, weekday_day_times, weekday_day_dates, weekdayDayClusters, weekdayDayClusterDays, weekdayDayClusterUniquePoints, weekdayDayClusterCentroids, numberOfClusters)


The clustering step sometimes creates multiple clusters that represent the same location. I fix this issue below by combining clusters that are very close (distance-wise) to each other.

The 'maxDistToCombine' parameter specifies that if two cluster centers are within 0.05 miles (264 feet, 0.08 km) of each other, then combine the clusters.

In [ ]:
maxDistToCombine = 0.05

# Returns mapping from point to cluster number
def combineSimilarClusters(clusters):
    
    clusterPoints = list(clusters)    
    combos = list()
    pointTaken = list()
    
    for i in range(0, len(clusterPoints)):
        combos.append(list())
        pointTaken.append(False)
    
    for i in range(0, len(clusterPoints)):       
        if pointTaken[i]:
            continue
            
        prevClusterPoint = clusterPoints[i]
        combos[i].append(i)
        pointTaken[i] = True
    
        for j in range(i+1, len(clusterPoints)):
            if pointTaken[j]:
                continue
                
            currClusterPoint = clusterPoints[j]
        
            distance = calculateDistance(prevClusterPoint[0], prevClusterPoint[1], currClusterPoint[0], currClusterPoint[1])
            
            if distance <= maxDistToCombine:
                combos[i].append(j)
                pointTaken[j] = True
                
    newClusters = list()
    
    for i in range(0, len(combos)):
        if len(combos[i]) > 0:
            comboList = list()
            for j in range(0, len(combos[i])):
                comboList.append(clusterPoints[combos[i][j]])
            newClusters.append(comboList)

    # Map each original point back to its cluster number
    pointToClusterNum = dict()
    
    for i in range(0, len(newClusters)):
        for j in newClusters[i]:
            pointToClusterNum[j] = i
    
    return (len(newClusters), pointToClusterNum)
    


Helper method for combining clusters. Find all the unique points associated with the to-be-combined clusters so that we can store all the points associated with the new cluster.

In [ ]:
def groupUniquePointsPerCluster(numClusters, pointToNewClusterNum, weekendClusterUniquePoints, weekdayDayClusterUniquePoints, weekdayEveningClusterUniquePoints):

    uniquePointsPerCluster = list()
    for i in range(0, numClusters):
        uniquePointsPerCluster.append(set())
        
    for point in weekendClusterUniquePoints.keys():
        clusterNum = pointToNewClusterNum[point]
        for newPoint in weekendClusterUniquePoints[point]:
            uniquePointsPerCluster[clusterNum].add(newPoint)

    for point in weekdayDayClusterUniquePoints.keys():
        clusterNum = pointToNewClusterNum[point]
        for newPoint in weekdayDayClusterUniquePoints[point]:
            uniquePointsPerCluster[clusterNum].add(newPoint)

    for point in weekdayEveningClusterUniquePoints.keys():
        clusterNum = pointToNewClusterNum[point]
        for newPoint in weekdayEveningClusterUniquePoints[point]:
            uniquePointsPerCluster[clusterNum].add(newPoint)

    return uniquePointsPerCluster

Helper method for combining clusters. Find all unique dates associated with the new cluster.

In [ ]:
def recalculateClusters(clusters, clusterDays, pointToNewClusterNum):
    
    newClusters = set()
    daysInClusters = dict()
    
    for cPoint in clusters.keys():
        cNum = pointToNewClusterNum[cPoint]
        newClusters.add(cNum)
        
        cDays = clusterDays[cPoint]
        
        if cNum not in daysInClusters:
            daysInClusters[cNum] = set()
        daysInClusters[cNum].update(cDays)
        
    return (list(newClusters), daysInClusters)
    

Kick off the combining of close clusters.

In [ ]:
(numNewClusters, pointToNewClusterNum) = combineSimilarClusters(set(list(weekendClusters.keys()) + list(weekdayDayClusters.keys()) + list(weekdayEveningClusters.keys())))

clusterUniquePoints = groupUniquePointsPerCluster(numNewClusters, pointToNewClusterNum, weekendClusterUniquePoints, weekdayDayClusterUniquePoints, weekdayEveningClusterUniquePoints)

(weekendClusters, weekendClusterDays) = recalculateClusters(weekendClusters, weekendClusterDays, pointToNewClusterNum) 
(weekdayDayClusters, weekdayDayClusterDays) = recalculateClusters(weekdayDayClusters, weekdayDayClusterDays, pointToNewClusterNum)
(weekdayEveningClusters, weekdayEveningClusterDays) = recalculateClusters(weekdayEveningClusters, weekdayEveningClusterDays, pointToNewClusterNum)

In [ ]:
print(numNewClusters)
for i in range(0, 4):
    print(list(clusterUniquePoints[i])[0])

Calculate bounding latitudes and longitudes (min/max of each) for each cluster based on the unique points associated with that cluster. This is important because when I'm trying to figure out if I was at one of these locations at a specific date/time, I can check to see if the current point is within the cluster boundaries (using a single lat/long point to represent the cluster is not useful because it's unlikely you will be at that exact same point, even if you are at the correct location).

In [ ]:
def calculateLocationBounds(uniquePointsPerCluster):

    clustersToBoundaries = dict()

    for i in range(0, len(uniquePointsPerCluster)):
        
        minLat = 0.0
        maxLat = 0.0
        minLong = 0.0
        maxLong = 0.0
    
        for uPoint in uniquePointsPerCluster[i]:
            currLat = float(uPoint[0])
            currLong = float(uPoint[1])
        
            if minLat == 0.0 or currLat < minLat:
                minLat = currLat
            
            if maxLat == 0.0 or currLat > maxLat:
                maxLat = currLat

            if minLong == 0.0 or currLong < minLong:
                minLong = currLong
            
            if maxLong == 0.0 or currLong > maxLong:
                maxLong = currLong
        
        # These are the boundaries
        currBoundary = (minLat, maxLat, minLong, maxLong)
        clustersToBoundaries[i] = currBoundary
        
    return clustersToBoundaries

clustersToBoundaries = calculateLocationBounds(clusterUniquePoints)


### Finding "Home" Cluster and Identifying associated "Home" Information

Supporting method to determine which cluster is 'home'. Uses a determination of how many unique days are represented in the cluster, as well as whether a cluster is prominent in both the weekend and weekday evening groups of points.

In [ ]:
# Return a reverse-sorted list of fractions of days I was at this location
def calculateFractions(clusters, clusterDays, numUniqueDates):
    fractionOfDays = dict()

    for clusterNum in clusters:
        numDaysInCluster = len(clusterDays[clusterNum])
        fraction = float(numDaysInCluster)/float(numUniqueDates)    
        fractionOfDays[fraction] = clusterNum
  
    reversedSortedKeys = list(reversed(sorted(fractionOfDays.keys())))
    return (reversedSortedKeys, fractionOfDays)


def calculateHomeInformation(weekendClusters, weekdayEveningClusters, weekendClusterDays, weekdayEveningClusterDays, clusterBoundings, numUniqueWeekendDates, numUniqueWeekdayEveningDates):
    homePoints = dict()

    (weekendSortedKeys, weekendFractions) = calculateFractions(weekendClusters, weekendClusterDays, numUniqueWeekendDates)
    (weekdayEveningSortedKeys, weekdayEveningFractions) = calculateFractions(weekdayEveningClusters, weekdayEveningClusterDays, numUniqueWeekdayEveningDates)
        
    for frac in weekendSortedKeys:
        if frac >= 0.5:
            clusterNum = weekendFractions[frac]
            
            homePoints[clusterNum] = dict()
            homePoints[clusterNum]["count"] = 1
            homePoints[clusterNum]["avg fraction"] = frac
            
        else:
            break

    for frac in weekdayEveningSortedKeys:
        if frac >= 0.5:
            clusterNum = weekdayEveningFractions[frac]

            if clusterNum in homePoints:
                currentAvgFraction = homePoints[clusterNum]["avg fraction"]
                currentCount = homePoints[clusterNum]["count"]

                homePoints[clusterNum]["avg fraction"] = (currentAvgFraction * float(currentCount) + frac)/float(currentCount + 1)
                homePoints[clusterNum]["count"] += 1

            else:
                homePoints[clusterNum] = dict()
                homePoints[clusterNum]["count"] = 1
                homePoints[clusterNum]["avg fraction"] = frac
        else:
            break

    highestHomeOccurrences = list()
    highestCount = 0
    
    lenOfHomePoints = len(homePoints)
    
    if lenOfHomePoints > 1:
        for (currCluster, theDict) in homePoints.items():
            theCount = theDict["count"]
            
            if theCount > highestCount:
                highestHomeOccurrences = list()
                highestHomeOccurrences.append(currCluster)
                highestCount = theCount
            elif theCount == highestCount:
                highestHomeOccurrences.append(currCluster)
                
    elif lenOfHomePoints == 1:
        highestHomeOccurrences.append(list(homePoints.keys())[0])
        
    lenOfHiHomePoints = len(highestHomeOccurrences)
    
    if lenOfHiHomePoints > 1:
        highestFraction = 0.0

        for cluster in highestHomeOccurrences:
            avgFrac = homePoints[cluster]["avg fraction"]
            if avgFrac > highestFraction:
                highestCluster = cluster
                highestFraction = avgFrac
    elif lenOfHiHomePoints == 1:
        highestCluster = highestHomeOccurrences[0]
    
    # Try to get most accurate bounding box before we return
    homePointInfo = homePoints[highestCluster]
    
    homePointInfo['bounding'] = clusterBoundings[highestCluster]
    homePointInfo['clusterNum'] = highestCluster
        
    # return the representative point and a dict that includes the count and bounding box
    # to get "home point", just calculate the midpoint between lats and midpoint between longs
    highestHomePoint = ((homePointInfo['bounding'][0] + homePointInfo['bounding'][1])/2.0, (homePointInfo['bounding'][2] + homePointInfo['bounding'][3])/2.0)
    return (highestHomePoint, homePointInfo)



Calculate and store information about home.

In [ ]:
(homePoint, homePointInfo) = calculateHomeInformation(weekendClusters, weekdayEveningClusters, weekendClusterDays, weekdayEveningClusterDays, clustersToBoundaries, numUniqueWeekendDates, numUniqueWeekdayEveningDates)

print("Home Point: " + str(homePoint))
print("Home Point Info: " + str(homePointInfo))

Create a data structure that will store information about the type of location a cluster represents.

In [ ]:
clusterLocationTypes = dict()
for i in range(0, numNewClusters):
    clusterLocationTypes[i] = set()
    
clusterLocationTypes[homePointInfo['clusterNum']].add('home')

For each point in the dataset when I was home, label with home's cluster number.

In [ ]:
# If appendLabel = True, will add the label if one already exists. Otherwise, will not add. 
def labelPoint(point, boundingPoints, infoDict, label, appendLabel):
    locationLabelExists = 'locationLabel' in infoDict
    
    if not appendLabel and locationLabelExists:
        return
    
    if pointWithinBoundingBox(point, boundingPoints):
        infoDict['locationLabel'] = label
        
for (date, dateDict) in sortedLocations.items():
    for (time, timeInfo) in dateDict['timeInfo'].items():
        labelPoint(timeInfo['point'], homePointInfo['bounding'], timeInfo, homePointInfo['clusterNum'], True)


Now that I know where home is, calculate the probability that I'm home for every hour of every day. High probabilities are used to calculated ranges during the day when I'm most likely to be home.

When I know high-probabability times I'm home, then the days when I'm not home at those times strongly indicate that I was out of town/away from home on that day.

This could get tricky for other peoples' data, since I'm very reliably home during the early morning hours (midnight - 4AM) and the very late night hours (10/11P - midnight). For people who are really only home during one of those periods, they may get some false positives from the algorithm that says they are away from home more often than they really are.

In [ ]:
# Calculate probability of being home during each hour for every day of the week
homePointsCount = dict()
homeCluster = homePointInfo['clusterNum']

for (date, dateDict) in sortedLocations.items():
    dayOfWeek = dateDict['dayOfWeek']
    
    if dayOfWeek not in homePointsCount:
        homePointsCount[dayOfWeek] = dict()
        
    for (time, timeInfo) in dateDict['timeInfo'].items():
        currHour = time[0:2]
        
        if currHour not in homePointsCount[dayOfWeek]:
            homePointsCount[dayOfWeek][currHour] = dict()
            homePointsCount[dayOfWeek][currHour]['totalPoints'] = 0
            homePointsCount[dayOfWeek][currHour]['homePoints'] = 0
        
        homePointsCount[dayOfWeek][currHour]['totalPoints'] += 1
        
        if 'locationLabel' in timeInfo and homeCluster == timeInfo['locationLabel']:
            homePointsCount[dayOfWeek][currHour]['homePoints'] += 1

# Calculate probabilities and store max probability per day of week
maxProb = dict()
for i in range(0, 7):
    maxProb[i] = 0.0

for (dayOfWeek, hourCounts) in homePointsCount.items():
    for (hour, pointsDict) in hourCounts.items():
        currFraction = float(pointsDict['homePoints'])/float(pointsDict['totalPoints'])
        pointsDict['fraction'] = currFraction
        if currFraction > maxProb[dayOfWeek]:
            maxProb[dayOfWeek] = currFraction

hoursInDay = [ '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', 
               '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23' ]

dayOfWeekRanges = dict()

for dayOfWeek in range(0, 7):
    hourCounts = homePointsCount[dayOfWeek]
    
    dayOfWeekRanges[dayOfWeek] = list()
    
    startRange = -1
    currHour = -1
    
    for hid in hoursInDay:
        hourInt = int(hid)
        
        if hourCounts[hid]['fraction'] >= (maxProb[dayOfWeek] - 0.2) :
            if startRange == -1:
                startRange = hourInt
                currHour = hourInt
            else:
                if (currHour + 1) == hourInt:
                    currHour = hourInt
                else:
                    if (currHour - startRange) > 1 or currHour == 23:
                        dayOfWeekRanges[dayOfWeek].append((startRange, currHour))
                    
                    startRange = hourInt
                    currHour = hourInt
        else:
            if startRange != -1:
                if (currHour - startRange) > 1 or currHour == 23:
                    dayOfWeekRanges[dayOfWeek].append((startRange, currHour))
                startRange = -1
                currHour = -1
                
    if startRange != -1 and currHour != -1:
        if (currHour - startRange) > 1 or currHour == 23:
            dayOfWeekRanges[dayOfWeek].append((startRange, currHour))
            
print(dayOfWeekRanges)
    

For each point in 'sortedLocations', store the distance from home at that time.

Identify dates when I was 'local', 'awayFromHome', or 'traveling'. Traveling means that for the specific day being analyzed, I was home during at least one of the high-confidence periods (stored in 'dayOfWeekRanges'), but not home in other period(s) in the same day. (My perhaps-erroneous assumption is that each day has more than one high-confidence period). Being home during one/some of the high-confidence periods, but not others, implies that I was either beginning my travels that day or returning from a trip that day.

In [ ]:
# For each hour in the day, record the distance from home to current location (1 per hour per day)
# If I was not home at least once in the high-confidence time periods, then treat this as a vacation day.

homeBounding = homePointInfo['bounding']

datesAway = list()
datesTraveling = list()
    
for (date, dateDict) in sortedLocations.items():
    dayOfWeek = dateDict['dayOfWeek']
        
    rangesForDay = dayOfWeekRanges[dayOfWeek]
    wasHomeInRange = list()
    rangeChecked = list()
    
    firstTimeDist = -1.0
    lastTimeDist = -1.0
    
    for i in range(0, len(rangesForDay)):
        wasHomeInRange.append(False)
        rangeChecked.append(False)
    
    # If I was home at any time during the high-confidence hours, I was not traveling    
    for (time, timeInfo) in dateDict['timeInfo'].items():  
        thisHour = int(time[0:2])
        distance = None
    
        for i in range(0, len(rangesForDay)):
            thisRange = rangesForDay[i]
            if thisHour >= thisRange[0] and thisHour <= thisRange[1]:
                rangeChecked[i] = True
                
                if 'locationLabel' in timeInfo and homeCluster == timeInfo['locationLabel']:
                    wasHomeInRange[i] = True
                    
                distance = 0.0
                break
        
        if distance == None:
            currPoint = timeInfo['point']
            #print(currPoint)
            #print(homePoint)
            distance = calculateDistance(currPoint[0], currPoint[1], homePoint[0], homePoint[1])
        
        timeInfo['distanceFromHome'] = distance
        

    for i in range(0, len(rangeChecked)):
        if not rangeChecked[i]:
            sortedTimes = sorted(dateDict['timeInfo'].keys())
            firstTimeDist = dateDict['timeInfo'][sortedTimes[0]]['distanceFromHome']
            lastTimeDist = dateDict['timeInfo'][sortedTimes[-1]]['distanceFromHome']
            
            if firstTimeDist <= 5.0 and lastTimeDist <= 5.0:
                rangeChecked[i] = True
                wasHomeInRange[i] = True
            
    numTrue = 0
    numFalse = 0
    for i in range(0, len(rangesForDay)):
        if wasHomeInRange[i]:
            numTrue += 1
        else:
            numFalse += 1
    
    if numTrue > 0:
        if numFalse > 0:
            # was traveling
            datesTraveling.append(date)
    else:
        # If no true, then was away
        datesAway.append(date)


Now I can determine (and label) which dates I was 'awayFromHome', 'traveling', or 'local'.

In [ ]:
for (date, dateInfo) in sortedLocations.items():
    if date in datesAway:
        dateInfo["dateDescription"] = "awayFromHome"
    elif date in datesTraveling:
        dateInfo["dateDescription"] = "traveling"
    else:
        dateInfo["dateDescription"] = "local"
        

Calculate the median radius of travel for when I'm local. Also, find the maximum distance I typically travel when I'm local (return home at the end of the day).

In [ ]:
maxDistancesFromHome = list()

for (date, dateInfo) in sortedLocations.items():
    if dateInfo["dateDescription"] == "local":
        maxDistance = -1
        for (eachTime, eachTimeInfo) in dateInfo["timeInfo"].items():
            distFromHome = eachTimeInfo['distanceFromHome']
            if distFromHome > maxDistance:
                maxDistance = distFromHome
                
        maxDistancesFromHome.append(maxDistance)
        
# Note: use median, because it actually reflects a true value.
medianHomeRadius = statistics.median(maxDistancesFromHome)
roundUpHomeRadius = math.ceil(medianHomeRadius)
maxLocalDistance = math.ceil(max(maxDistancesFromHome))

print("Median: " + str(medianHomeRadius))
print("Boundary: " + str(roundUpHomeRadius))
print("Max local distance: " + str(maxLocalDistance))


### Working with other, non-Home clusters

Check each dataset point to determine if I was at one of the identified cluster locations. If so, label that point with the cluster number.

Also, store cluster-related information in data structures that can be easily used to look up information later.

In [ ]:
# Look at all the non-home points and get the bounding boxes for them
def addLocationInformation(clusters, timeOfDayType, parentDict, clustersToBoundaries, boundingToCluster, homeAddress, homePointInfo):    
    homeCluster = homePointInfo['clusterNum']
    
    for clusterNum in clusters:
        clusterBounding = clustersToBoundaries[clusterNum]
        # Only store if it's not related to the home point
        if clusterNum != homeCluster:
            if clusterNum not in parentDict:
                locationDict = dict()
                locationDict['timeOfDayType'] = set()
                parentDict[clusterNum] = locationDict
                locationDict['locationType'] = set()
                
                boundingToCluster[tuple(clusterBounding)] = clusterNum
                    
            parentDict[clusterNum]['timeOfDayType'].add(timeOfDayType)            
            
clusterLocationInformation = dict()
boundingToClusterMap = dict()

addLocationInformation(weekendClusters, "weekend", clusterLocationInformation, clustersToBoundaries, boundingToClusterMap, homePoint, homePointInfo)
addLocationInformation(weekdayEveningClusters, "weekdayEvening", clusterLocationInformation, clustersToBoundaries, boundingToClusterMap, homePoint, homePointInfo)
addLocationInformation(weekdayDayClusters, "weekdayDay", clusterLocationInformation, clustersToBoundaries, boundingToClusterMap, homePoint, homePointInfo)

Per cluster, store the unique dates associated with that cluster in my dataset.

Also, store the type of day that was ('awayFromHome', 'traveling', 'local') to help determine what type of location that cluster is.

In [ ]:
clusterUniqueDays = dict()

# Keep track of # unique days for this location
for cluster in clustersToBoundaries.keys():
    if cluster != homeCluster:
        clusterUniqueDays[cluster] = set()

for (date, dateInfo) in sortedLocations.items():
    dateDescription = dateInfo['dateDescription']
    
    for (time, timeInfo) in dateInfo['timeInfo'].items():
        point = timeInfo['point']
        for bounding in boundingToClusterMap.keys():
            storedCluster = boundingToClusterMap[bounding]

            if pointWithinBoundingBox(point, bounding):
                clusterUniqueDays[storedCluster].add(date)

                timeInfo['locationLabel'] = storedCluster
                clusterLocationInformation[storedCluster]['locationType'].add(dateDescription)
                break


Use the dates I already know about in 'datesAway' and 'datesTraveling' to identify cluster locations that are either 'awayFromHome' or 'local'. There should no longer be a 'traveling' type of cluster, since 'traveling' is just a label for a day when it appears that I'm 'local' for part of the day, but 'local' for another part of the day.

In [ ]:
# Determine if "one-time location" or if it's a consistent location
# locationType == 'traveling' or 'local'

clustersAway = set()
clustersTraveling = set()
clustersLocal = set()

for (cluster, listOfDays) in clusterUniqueDays.items():
    numDaysForCluster = len(listOfDays)    
    addedAlready = False
    
    for locationDay in listOfDays:
        # See if any of these locations are found on the days I was away/traveling
        if locationDay in datesAway:
            clustersAway.add(cluster)
            addedAlready = True
    
        if locationDay in datesTraveling:
            clustersTraveling.add(cluster)
            addedAlready = True
        
        if not addedAlready:
            clustersLocal.add(cluster)

# Remove anything in 'away' set from 'traveling' set
for locationCluster in list(clustersAway):
    if locationCluster in clustersTraveling:
        clustersTraveling.remove(locationCluster)

# Remove anything from 'local' set from 'traveling' set
for locationCluster in list(clustersLocal):
    if locationCluster in clustersTraveling:
        clustersTraveling.remove(locationCluster)

# Determine if clusters in traveling are actually local or away by comparing distance with [max] radius
for cluster in list(clustersTraveling):
    clusterBoundary = clustersToBoundaries[cluster]
    midPoint = ( (clusterBoundary[0] + clusterBoundary[1])/2.0, (clusterBoundary[2] + clusterBoundary[3])/2.0 ) 
    clusterDist = calculateDistance(midPoint[0], midPoint[1], homePoint[0], homePoint[1])
        
    if clusterDist <= roundUpHomeRadius:
        clustersLocal.add(cluster)
    elif clusterDist <= maxLocalDistance:
        clustersLocal.add(cluster)
    else:
        clustersAway.add(cluster)

    clustersTraveling.remove(cluster)

        
print("===========================")
print("AWAY")
for cl in clustersAway:
    print(str(clustersToBoundaries[cl]))
print("===========================")
print("TRAVELING")
for cl in clustersTraveling:
    print(str(clustersToBoundaries[cl]))
print("===========================")
print("LOCAL")
for cl in clustersLocal:
    print(str(clustersToBoundaries[cl]))
        

Using the dataset, store days of week and times when I was at each cluster location.

In [ ]:
localClustersCount = dict()

for clusterNum in clustersLocal:
    clusterInfo = clusterLocationInformation[clusterNum]
    
    thisClusterCount = dict()

    for (date, dateDict) in sortedLocations.items():
        dayOfWeek = dateDict['dayOfWeek']
            
        for (time, timeInfo) in dateDict['timeInfo'].items():
            currHour = time[0:2]

            if 'locationLabel' in timeInfo and timeInfo['locationLabel'] == clusterNum:
                if dayOfWeek not in thisClusterCount:
                    thisClusterCount[dayOfWeek] = dict()

                if currHour not in thisClusterCount[dayOfWeek]:
                    thisClusterCount[dayOfWeek][currHour] = dict()
                    thisClusterCount[dayOfWeek][currHour]['localPoints'] = 0
                    
                thisClusterCount[dayOfWeek][currHour]['localPoints'] += 1       

    localClustersCount[clusterNum] = thisClusterCount


Using the number of days and hours seen at each place, figure out what type of local location the cluster is.

In [ ]:
def isWeekendLocation(dowInfo):
    days = list(dowInfo.keys())
    if 5 in days and 6 in days:
        return True
    
    
def getWeekdayLocationType(dowInfo):
    
    days = list(dowInfo.keys())
    weekdays = range(0, 5)
    
    for i in weekdays:
        if i not in days:
            return (False, '')
        
    weekdayHours = 0.0
    weekendHours = 0.0

    # Figure out avg number of hours for weekday vs. weekends (if any)
    for (dow, pointsInfo) in dowInfo.items():
        if dow in weekdays:
            weekdayHours += float(len(pointsInfo.keys()))
        else:
            weekendHours += float(len(pointsInfo.keys()))

    weekdayAvg = weekdayHours/5.0
    weekendAvg = weekendHours/2.0
            
    if len(days) == 5 and weekdayAvg >= 5.0:
        return (True, 'work')
    if len(days) > 5:
        if weekdayAvg >= 5.0 and weekendAvg <= 4.0:
            return (True, 'work')
        else:
            return (False, 'weekday: ' + str(weekdayAvg) + ", weekend: " + str(weekendAvg))
    
    return (False, '')

Try to figure out the type of local location for each cluster (may not always be possible).

'work' is self-explanatory.
'weekend' is a location where I'm only found on weekends.
'sameWeekday' is a location where I'm only found on a single day of the week.

In [ ]:
for (cl, dowInfo) in localClustersCount.items():
    daysOfWeek = list(dowInfo.keys())
    numDaysOfWeek = len(daysOfWeek)
    
    if numDaysOfWeek == 1:
        clusterLocationTypes[cl].add('sameDayPerWeek')
    else:
        if numDaysOfWeek == 2:
            if isWeekendLocation(dowInfo):
                clusterLocationTypes[cl].add('weekend')
       
        elif numDaysOfWeek >= 5:
            (isWeekday, info) = getWeekdayLocationType(dowInfo)
            if isWeekday:
                if info == 'work':
                    clusterLocationTypes[cl].add('work')
    

for (clusterNum, types) in clusterLocationTypes.items():
    if len(types) > 0:
        print("Cluster: " + str(clusterNum) + " (" + str(clustersToBoundaries[clusterNum]) + ") has the following type(s): " + str(types))


Now, using these data structures:
  - sortedLocations
  - clusterLocationTypes


you should have a complete picture of:
  - Where/how far away from home you were at specific time on a specific day
  - Whether you were at a significant location (cluster you calculated)
  - If you were at a cluster location, whether that cluster location is a special type of location (i.e., work, home, weekend, or sameDayPerWeek).